In [1]:
# Cell 1 – imports & config
import gc
import json
import numpy as np
import pandas as pd
import xgboost as xgb

META_PATH = '../data/outputs/dataset_meta.json'
TRAIN_RANK_PATH = '../data/outputs/train_rank.parquet'
VALID_RANK_PATH = '../data/outputs/valid_rank.parquet'
GROUP_TRAIN_PATH = '../data/outputs/groups_train.npy'
GROUP_VALID_PATH = '../data/outputs/groups_valid.npy'


In [2]:
# Cell 2 – load train/valid ranking data
with open(META_PATH) as f:
    meta = json.load(f)

feature_cols = meta['model_features']

train_df = pd.read_parquet(TRAIN_RANK_PATH)
valid_df = pd.read_parquet(VALID_RANK_PATH)

# REMOVE FEATURES – must match LGBM
remove_features = [
    # 'value',
    # 'age_bucket',
    # 'customer_total_purchases',
    # 'customer_unique_articles',
    # 'cust_purchases_1w',
    # 'cust_purchases_4w',
    # 'article_total_purchases',
    # 'article_unique_customers',
    #'club_member_status',
    #'fashion_news_frequency',
    #'age',
    #'postal_code',
    #'product_code',
    #'product_type_no',
    #'product_group_name',
    #'graphical_appearance_no',
    #'colour_group_code',
    #'perceived_colour_value_id',
    #'perceived_colour_master_id',
    #'department_no',
    #'index_code',
    #'index_group_no',
    #'section_no',
    #'garment_group_no',
    # 'article_mean_price',
    # 'customer_mean_price',
    # 'article_mean_age',
    # 'index_group_match',
    # 'product_code_match',
    # 'days_since_last_purchase',
    # 'customer_days_since_last_purchase',
    # 'price_sensitivity',
    # 'age_sensitivity',
    # 'window_type_code',
]

drop_cols = [c for c in remove_features if c in train_df.columns]
if drop_cols:
    print("Dropping from TRAIN/VALID:", drop_cols)
    train_df = train_df.drop(columns=drop_cols)
    valid_df = valid_df.drop(columns=drop_cols, errors='ignore')

# Update feature_cols identically
feature_cols = [f for f in feature_cols if f not in remove_features]

train_group = np.load(GROUP_TRAIN_PATH)
valid_group = np.load(GROUP_VALID_PATH)

# Float32 conversion for XGBoost
for c in feature_cols:
    if pd.api.types.is_float_dtype(train_df[c]):
        train_df[c] = train_df[c].astype('float32')
        valid_df[c] = valid_df[c].astype('float32')

train_df['label'] = train_df['label'].astype('float32')
valid_df['label'] = valid_df['label'].astype('float32')

print("Train rows:", len(train_df))
print("Valid rows:", len(valid_df))
print("Features:", len(feature_cols))
print("Groups train:", len(train_group), "valid:", len(valid_group))

gc.collect()

#40s


Train rows: 8180082
Valid rows: 253714158
Features: 34
Groups train: 294983 valid: 1371980


22

In [3]:
# %%
# Cell 3 – filter validation groups with ≥1 positive (match LightGBM)

print("\nFiltering validation set to groups with at least 1 positive...")

# Use original valid_group from disk (same as LGBM)
orig_valid_group = np.load(GROUP_VALID_PATH)
group_bounds = np.insert(np.cumsum(orig_valid_group), 0, 0)[:-1]
group_has_positive = np.add.reduceat(valid_df['label'].values, group_bounds) > 0

valid_group = orig_valid_group[group_has_positive]
row_mask = np.repeat(group_has_positive, orig_valid_group)

# Apply mask to validation dataframe
valid_df = valid_df[row_mask].reset_index(drop=True)

print("Filtered valid groups:", len(valid_group))
print("Filtered valid rows:", len(valid_df))

gc.collect()



Filtering validation set to groups with at least 1 positive...
Filtered valid groups: 27802
Filtered valid rows: 8372080


11

In [4]:
# %%
# Cell 4 – always train XGBoost ranker and save model

MODEL_PATH = "../data/outputs/xgb_ranker.model"

dtrain = xgb.DMatrix(train_df[feature_cols], label=train_df['label'])
dvalid = xgb.DMatrix(valid_df[feature_cols], label=valid_df['label'])

dtrain.set_group(train_group)
dvalid.set_group(valid_group)

params = {
    'objective': 'rank:ndcg',
    'eval_metric': 'ndcg@12',
    'eta': 0.15,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'hist',
    'random_state': 42,
}

num_round = 3000
early_stopping = 100

evals_result = {}
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=num_round,
    evals=[(dtrain,'train'), (dvalid,'valid')],
    early_stopping_rounds=early_stopping,
    evals_result=evals_result,
    verbose_eval=50,
)

print("Best iteration:", bst.best_iteration)
print("Best valid ndcg@12:", evals_result['valid']['ndcg@12'][bst.best_iteration])

# Always save the trained model
bst.save_model(MODEL_PATH)
print("Model saved to:", MODEL_PATH)

# ==========================================================
# Save training log to TXT
# ==========================================================
log_path = "../data/outputs/xgb_training_log.txt"

train_log = evals_result.get('train', {}).get('ndcg@12', [])
valid_log = evals_result.get('valid', {}).get('ndcg@12', [])

max_len = min(len(train_log), len(valid_log))

with open(log_path, "w") as f:
    f.write("iter,train_ndcg12,valid_ndcg12\n")
    for i in range(max_len):
        f.write(f"{i},{train_log[i]},{valid_log[i]}\n")

print("Saved training log to:", log_path)

# Free memory
del dtrain, dvalid, bst
gc.collect()

# 10 min

[0]	train-ndcg@12:0.77550	valid-ndcg@12:0.13621
[50]	train-ndcg@12:0.79589	valid-ndcg@12:0.15802
[100]	train-ndcg@12:0.80283	valid-ndcg@12:0.16185
[150]	train-ndcg@12:0.80692	valid-ndcg@12:0.16427
[200]	train-ndcg@12:0.80994	valid-ndcg@12:0.16577
[250]	train-ndcg@12:0.81213	valid-ndcg@12:0.16662
[300]	train-ndcg@12:0.81423	valid-ndcg@12:0.16724
[350]	train-ndcg@12:0.81586	valid-ndcg@12:0.16742
[400]	train-ndcg@12:0.81742	valid-ndcg@12:0.16795
[450]	train-ndcg@12:0.81889	valid-ndcg@12:0.16793
[500]	train-ndcg@12:0.82036	valid-ndcg@12:0.16803
[550]	train-ndcg@12:0.82168	valid-ndcg@12:0.16799
[600]	train-ndcg@12:0.82297	valid-ndcg@12:0.16803
[636]	train-ndcg@12:0.82394	valid-ndcg@12:0.16821
Best iteration: 536
Best valid ndcg@12: 0.16836042801452428
Model saved to: ../data/outputs/xgb_ranker.model
Saved training log to: ../data/outputs/xgb_training_log.txt


/tmp/ipykernel_38059/2300016569.py:41: UserWarning: [14:59:21] WARNING: /workspace/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  bst.save_model(MODEL_PATH)


28

In [5]:
# %%
# Cell 5 – always load saved model and perform inference with progress logging

MODEL_PATH = "../data/outputs/xgb_ranker.model"

print("Loading stored model:", MODEL_PATH)
bst = xgb.Booster()
bst.load_model(MODEL_PATH)
print("Model loaded, best iteration:", bst.best_iteration)

FEATURES_PATH = "../data/outputs/features_week=20200922.parquet"

data = pd.read_parquet(FEATURES_PATH, columns=['customer_id','article_id'] + feature_cols)

# enforce correct dtypes before ANY further processing
data['customer_id'] = data['customer_id'].astype('int64')
data['article_id'] = data['article_id'].astype('int32')

# Ensure float32 for XGBoost features
for c in feature_cols:
    if pd.api.types.is_float_dtype(data[c]):
        data[c] = data[c].astype('float32')

BATCH = 2_000_000
n_rows = len(data)
scores = np.empty(n_rows, dtype=np.float32)

print(f"Running inference on {n_rows:,} rows...")

for start in range(0, n_rows, BATCH):
    end = min(start + BATCH, n_rows)

    dmatrix_batch = xgb.DMatrix(data.iloc[start:end][feature_cols])
    scores[start:end] = bst.predict(
        dmatrix_batch,
        iteration_range=(0, bst.best_iteration + 1),
    ).astype(np.float32)

    del dmatrix_batch
    gc.collect()

    print(f"Predicted rows {start:,} to {end:,} / {n_rows:,}")

data['score'] = scores
del scores
gc.collect()

print("Final scored rows:", len(data))

# 13 min

Loading stored model: ../data/outputs/xgb_ranker.model
Model loaded, best iteration: 536


/tmp/ipykernel_38059/1173781005.py:8: UserWarning: [15:03:06] WARNING: /workspace/src/c_api/c_api.cc:1511: Unknown file format: `model`. Using UBJSON (`ubj`) as a guess.
  bst.load_model(MODEL_PATH)


Running inference on 250,982,495 rows...
Predicted rows 0 to 2,000,000 / 250,982,495
Predicted rows 2,000,000 to 4,000,000 / 250,982,495
Predicted rows 4,000,000 to 6,000,000 / 250,982,495
Predicted rows 6,000,000 to 8,000,000 / 250,982,495
Predicted rows 8,000,000 to 10,000,000 / 250,982,495
Predicted rows 10,000,000 to 12,000,000 / 250,982,495
Predicted rows 12,000,000 to 14,000,000 / 250,982,495
Predicted rows 14,000,000 to 16,000,000 / 250,982,495
Predicted rows 16,000,000 to 18,000,000 / 250,982,495
Predicted rows 18,000,000 to 20,000,000 / 250,982,495
Predicted rows 20,000,000 to 22,000,000 / 250,982,495
Predicted rows 22,000,000 to 24,000,000 / 250,982,495
Predicted rows 24,000,000 to 26,000,000 / 250,982,495
Predicted rows 26,000,000 to 28,000,000 / 250,982,495
Predicted rows 28,000,000 to 30,000,000 / 250,982,495
Predicted rows 30,000,000 to 32,000,000 / 250,982,495
Predicted rows 32,000,000 to 34,000,000 / 250,982,495
Predicted rows 34,000,000 to 36,000,000 / 250,982,495
Pred

In [6]:
# Cell 6 – top12 per customer
data['customer_id'] = data['customer_id'].astype('int64')

data = data.sort_values(['customer_id','score'], ascending=[True,False])
top12 = data.groupby('customer_id', group_keys=False).head(12)

top12['article_id_str'] = top12['article_id'].astype(str).str.zfill(10)

pred_df = (
    top12.groupby('customer_id')['article_id_str']
         .apply(lambda x: ' '.join(x))
         .reset_index()
         .rename(columns={'customer_id':'customer_id_int',
                          'article_id_str':'prediction'})
)

pred_df['customer_id_int'] = pred_df['customer_id_int'].astype('int64')

del top12, data
gc.collect()

print("Predictions:", len(pred_df))

# 8 min 20s


/tmp/ipykernel_38059/2283025980.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top12['article_id_str'] = top12['article_id'].astype(str).str.zfill(10)


Predictions: 1371980


In [7]:
# Cell 7 – merge with sample + fallback

# Must match preprocessing pipeline
def hex16_to_int(s):
    return np.int64(np.uint64(int(s[-16:], 16)))

sample = pd.read_csv('../data/input_data/sample_submission.csv')

# Convert customer_id using identical hashing
sample['customer_id_int'] = sample['customer_id'].apply(hex16_to_int)

# Drop any existing prediction column (same as LGBM)
sample = sample.drop(columns=['prediction'], errors='ignore')

# pred_df already contains correct int64 customer_id from features
pred_df['customer_id_int'] = pred_df['customer_id_int'].astype('int64')

# Merge exactly like LGBM
sub = sample.merge(pred_df, how='left', on='customer_id_int')

# Diagnostics
print("Predictions matched:", sub['prediction'].notna().sum())
print("Rows after merge:", len(sub))

del sample
gc.collect()

gp = json.load(open('../data/outputs/general_pred_str.json'))
fallback_str = gp['general_pred_str']
fallback_items = fallback_str.split()

sub['prediction'] = sub['prediction'].fillna(fallback_str)


Predictions matched: 1371980
Rows after merge: 1371980


In [8]:
# Cell 8 – pad/trim to 12
def pad_to_12(pred):
    items = pred.split()
    if len(items) >= 12:
        return ' '.join(items[:12])
    used = set(items)
    for art in fallback_items:
        if art not in used:
            items.append(art)
            used.add(art)
        if len(items) == 12:
            break
    return ' '.join(items)

sub['prediction'] = sub['prediction'].apply(pad_to_12)

out_path = '../data/submission/xgboost_ranker_submission.csv'
sub[['customer_id','prediction']].to_csv(out_path, index=False)
print("Saved:", out_path)
sub.head()


Saved: ../data/submission/xgboost_ranker_submission.csv


,customer_id,customer_id_int,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,6883939031699146327,0568601043 0568601044 0568601006 0568601007 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-7200416642310594310,0673677002 0918522001 0448509014 0158340001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,-6846340800584936,0794321007 0794321011 0794321008 0851400001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,-94071612138601410,0794321011 0804992017 0805000001 0730683050 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,-283965518499174310,0730683050 0896152002 0730683062 0791587015 08...


In [9]:
# Cell 9 – feature importance
imp = bst.get_score(importance_type='gain')

name_map = {f"f{i}": name for i,name in enumerate(feature_cols)}
items = [(name_map.get(k,k), v) for k,v in imp.items()]

for n,v in sorted(items, key=lambda x: -x[1]):
    print(n, v)

gc.collect()


product_code_match 106.54669189453125
value 106.06864166259766
window_type_code 91.05266571044922
garment_group_no 32.50369644165039
days_since_last_purchase 23.152313232421875
index_group_match 20.583600997924805
product_group_name 20.16489028930664
article_total_purchases 16.43085289001465
section_no 15.574021339416504
cust_purchases_1w 15.44941234588623
product_type_no 14.622406005859375
article_unique_customers 13.07671070098877
product_code 12.876992225646973
article_mean_price 12.628988265991211
department_no 11.702217102050781
colour_group_code 10.916023254394531
index_code 10.668539047241211
age_bucket 10.273480415344238
graphical_appearance_no 9.833288192749023
index_group_no 9.753656387329102
customer_mean_price 9.258064270019531
article_mean_age 9.063455581665039
customer_unique_articles 8.482888221740723
customer_days_since_last_purchase 8.455252647399902
age 8.193224906921387
price_sensitivity 7.996027946472168
perceived_colour_value_id 7.883965492248535
age_sensitivity 7.

19

In [10]:
# Cell 10 - OPTIONAL – Compute offline MAP@12 on 2020-09-16 validation set

import numpy as np
import pandas as pd

# 1. Extract ground truth for validation week
# valid_df already contains labels for week 20200916 after filtering groups
gt = valid_df[valid_df['label'] == 1][['customer_id', 'article_id']].copy()
gt['article_id_str'] = gt['article_id'].astype(str).str.zfill(10)

# Map each customer → set of purchased items
gt_dict = gt.groupby('customer_id')['article_id_str'].apply(set).to_dict()

# 2. Predictions (after your top-12 grouping)
# Should be: customer_id_int, prediction (10-digit strings)
pred = pred_df.copy()
pred['pred_list'] = pred['prediction'].str.split()

# Convert ID type if needed
pred['customer_id'] = pred['customer_id_int'].astype('int64')

# ---------------------------------------------------------
# 3. Compute MAP@12
# ---------------------------------------------------------

def apk(actual_set, predicted_list, k=12):
    """Average Precision@k for a single user."""
    if len(predicted_list) > k:
        predicted_list = predicted_list[:k]
    score = 0.0
    hits = 0
    for i, p in enumerate(predicted_list, start=1):
        if p in actual_set:
            hits += 1
            score += hits / i
    if not actual_set:
        return 0.0
    return score / min(len(actual_set), k)

# Apply APK@12 for each customer
scores = []
for cid, row in pred.iterrows():
    user = row['customer_id']
    actual = gt_dict.get(user, set())
    pred_list = row['pred_list']
    scores.append(apk(actual, pred_list, k=12))

map12 = np.mean(scores)

print("=====================================================")
print("Offline MAP@12 (Validation Week 20200916):", map12)
print("=====================================================")


Offline MAP@12 (Validation Week 20200916): 0.01351311158540974
